![Customer_Churn_Header](https://raw.githubusercontent.com/satishgunjal/images/master/Customer_Churn_Header.png)
<sub><sup>Img Source: NicePNG</sup></sub>

# Index
* [Business Problem](#1)
* [Machine Learning Problem](#2)
* [Approach](#3)
* [Import Libraries](#4)
* [Data Collection](#5)
* [Exploratory Data Analysis](#6)
  - [Understanding the Data](#7)
  - [Data Visualization & Analysis](#8)  
  - [Data Cleaning & Preprocessing](#9)
    - [Feature Selection](#10)
    - [Feature Encoding](#11)
    - [Transform Categorical Values](#12)
	- [Scaling & Normalization](#13)
* [Train Test Split](#14)
* [Modeling](#15)
  - [Model Building](#16)
  - [Model Training](#17)
  - [Model Evaluation](#18)
* [Conclusion](#19)
* [Reference](#20)


# Business Problem <a id ="1"></a>
Based on the available data for the telco customers, predict customer behavior and develop focused customer retention programs.

# Machine Learning Problem <a id ="2"></a>
In order to develop the customer retention program it's important to understand the biggest contributor to customer churning. We will be able to identify those features using exploratory data analysis. We will also build the model which can predict whether that customer churn or not. So based on our EDA analysis the company can design the proactive program that will avoid all those factors which leads to customer attrition and using our ML model they can periodically check whether the customer can leave or not and accordingly take measures to retain the customer.

# Approach <a id ="3"></a>
After you clearly define the Machine Learning problem it's time to finalize the approach to achieve it. There are many algorithms, tools and libraries to choose from. In this case we are going to build Artificial Neural Network (ANN) to predict the customer attrition, also known as customer churn, customer turnover, or customer defection. We will use TensorFlow deep learning framework to build our model.

Customer churn (rate) is the measure of the number of individuals or items moving out of a collective group over a specific period. It is one of two primary factors that determine the steady-state level of customers a business will support. 

Note: It's not required to use ANN for this problem you can also get very good results using Logistic Regression.

# Import Libraries <a id ="4"></a>
As per our final approach lets import all the necessary libraries that we will use over the course of this project.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

# Data Collection <a id ="5"></a>
Our data is in tabular format and source is CSV file. Since our data is available on Kaggle we don't have to do anything extra for collecting the data, we can go ahead and load the data using panda's 'read_csv' function.

In [ ]:
df = pd.read_csv("/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")
print(f'No of rows/training examples: {df.shape[0]}, No of columns/features: {df.shape[1]}')
df.sample(5)


# Exploratory Data Analysis <a id ="6"></a>

## Understanding the Data <a id ="7"></a>
* Data set contains 7,043 customers and 21 data points on each customer. 
* Each row represents a customer, each column contains the customer’s attributes.
* Column Churn contain the class labels value, 'Yes' means customer will churn else 'No'
* Customer attributes/features are as below

| Column Name | Description |
|:---|:--- |
| customerID | Customer ID |
| gender | Whether the customer is a male or a female |
| SeniorCitizen | Whether the customer is a senior citizen or not (1, 0) |
| Partner | Whether the customer has a partner or not (Yes, No) |
| Dependents | Whether the customer has dependents or not (Yes, No) |
| tenure | Number of months the customer has stayed with the company |
| PhoneService | Whether the customer has a phone service or not (Yes, No) |
| MultipleLines | Whether the customer has multiple lines or not (Yes, No, No phone service) |
| InternetService | Customer’s internet service provider (DSL, Fiber optic, No) |
| OnlineSecurity | Whether the customer has online security or not (Yes, No, No internet service)
| OnlineBackup | Whether the customer has online backup or not (Yes, No, No internet service) |
| DeviceProtection | Whether the customer has device protection or not (Yes, No, No internet service) |
| TechSupport | Whether the customer has tech support or not (Yes, No, No internet service) |
| StreamingTV | Whether the customer has streaming TV or not (Yes, No, No internet service) |
| StreamingMovies | Whether the customer has streaming movies or not (Yes, No, No internet service) |
| Contract | The contract term of the customer (Month-to-month, One year, Two year) |
| PaperlessBilling | Whether the customer has paperless billing or not (Yes, No) |
| PaymentMethod | The customer’s payment method (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic)) |
| MonthlyCharges | The amount charged to the customer monthly |
| TotalCharges | The total amount charged to the customer |

## Data Visualization & Analysis <a id ="8"></a>
* In this step we will try to visualize the data based on important features.
* Most important loss is losing the loyal customers. Using feature 'tenure' we can explore such customers.
* We will use histogram to plot the tenure and whether customer leaving or not.

Let review the statistics on the current data 

In [ ]:
df.describe()

As you can see from above data that tenure and MonthlyCharges values differ by order of magnitude, we will have to address this problem and normalize the data before modeling. Also, it will be interesting to see the effect of MonthlyCharges and tenure on the customer churn.
Now let's check for NA or missing values.

In [ ]:
df.isna().sum()

This is wonderful sign, there are no missing values in the data. One of the most important data is churn status of each customer. Let's print the number of customer  that churned(Yes) and not churned(No)

In [ ]:
# Count of customer that will churn
cust_churn_yes_count = df[df.Churn == 'Yes'].shape[0]
# Count of customer that will not churn (retain)
cust_churn_no_count = df[df.Churn == 'No'].shape[0]

# Percentage of customer that will churn
cust_churn_yes_percent = round((cust_churn_yes_count / (cust_churn_yes_count + cust_churn_no_count) * 100),2)
# Percentage of customer that will not churn (retain)
cust_churn_no_percent = round((cust_churn_no_count / (cust_churn_yes_count + cust_churn_no_count) * 100 ),2)

plt.figure(figsize=(10,6))
ax = sns.countplot(df['Churn'])
ax.set_title(f'{cust_churn_yes_percent} % ({cust_churn_yes_count} nos) of cust will churn & {cust_churn_no_percent} % ({cust_churn_no_count} nos) of cust will retain')

As you can see we have imbalanced data, most of the data contains class label 'No' means customer will remain with the company. If out of all the available data 73.46% customers are going to stay with the company, then for any given input we can say with 73% accuracy that the customer will stay with the company. So our first objective is to get the model's accuracy more than 73%, which is better than just doing the guess work.

Note that it's extremely important to get good quality balanced data so that the model will be generalized well enough to perform better in real life scenario. In our case we have to go ahead with whatever data is available and make the best out of it.

Now let's look at the effect of other available features on the customer retention.

In [ ]:
plt.figure(figsize=(10,6))
ax = sns.countplot(x= 'gender', hue='Churn', data=df)
ax.set_title(f'Effect of Gender on customer churn')

As you can see customer gender doesn't seem to affecting churn count. Since we have separate columns for senior citizens let check the churn rate for them.

In [ ]:
plt.figure(figsize=(10,6))
ax = sns.countplot(x= 'SeniorCitizen', hue='Churn', data=df)
ax.set_title(f'SeniorCitizens have higher customer churn')
plt.xlabel('SeniorCitizens(0: No, 1: Yes)')

As we can see from above graph that senior citizens though being few in number have higher customer churn percentage. Now let check the effect of internet service on churn count.

In [ ]:
plt.figure(figsize=(10,6))
ax = sns.countplot(x= 'InternetService', hue='Churn', data=df)
ax.set_title(f'Effect of internet service on customer churn')

As we can see from above graph that customers who are using DSL internet service tends to be more loyal as compare to fiber optic using customers. There can be many reasons for this and an accurate conclusion can be drawn by analyzing DSL and fiber optic data in detail. So my suggestion to the company is to find the shortcoming in the DSL service and keep doing what they're doing with fiber optic service. Also note that if customers are not using nay internet service then churn count reduces even further. So over all the company should have a look at the internet service offerings and try to fix bring down the shortcomings. 

Now let's have a look at the effect of tenure on customer churn.

In [ ]:
tenure_churn_yes = df[df.Churn == 'Yes'].tenure
tenure_churn_no = df[df.Churn == 'No'].tenure

plt.figure(figsize=(10,6))
plt.hist([tenure_churn_yes, tenure_churn_no], color=['orange', 'blue'], label= ['Leaving', 'Staying'])
plt.xlabel('Tenure(Months)')
plt.ylabel('No of Customers')
plt.title('Customers Staying vs Leaving Based on Tenure')
plt.legend()

From above histogram it's clear that more number of customers tend to stay with the company after the first 10 months of service. So the company can concentrate more on the new customer and their initial experience with the service to increase the customer retention.

Let also visualize the customer churn with respect to monthly charges.

In [ ]:
monthly_charges_churn_yes = df[df.Churn == 'Yes'].MonthlyCharges
monthly_charges_churn_no = df[df.Churn == 'No'].MonthlyCharges


plt.figure(figsize=(10,6))
plt.hist([monthly_charges_churn_yes, monthly_charges_churn_no], color=['orange', 'blue'], label= ['Leaving', 'Staying'])
plt.xlabel('Monthly Charges')
plt.ylabel('No of Customers')
plt.title('Customers Staying vs Leaving Based on Monthly Charges')
plt.legend()

As you see from above histogram for lowest monthly rates(between 20 and 40) highest is the customer retention. But customer attrition increases for monthly charges between 70 and 100. So the company can reduce the monthly charges or add more value to the service for the same price to increase the customer retention.

## Data Cleaning & Preprocessing <a id ="9"></a>
Data visualization gave us idea about the features which are affecting the customer churn. Now let's clean and precess the data to make it ready for modeling.

### Feature Selection <a id ="10"></a>
It's not necessary that all the given features are going to help predict customer churn. In our case customer id is one of that feature which will not add any value to our analysis or the model's accuracy. So let's remove the customer id columns from the dataset. Note that whenever I make major change in dataset I create new pandas dataframe and increase the number by 1.

In [ ]:
df1 = df.drop('customerID', axis = 'columns')
df1.shape # Print the shape of new dataframe

Now our dataset contains 7043 training examples and 20 features. Now let's review the data types of the features.

### Feature Encoding <a id ="11"></a>
* One of the important task is to make sure that each feature has datatype as per the data it represents. In this case 'MonthlyCharge' feature has float64 datatype but 'TotalCharges' has 'object' data type. 
* We will first print the datatype of all the columns and depending on the value it contains we will use either label or one hot encoding to convert the features to numeric format

In [ ]:
df1.dtypes

Let check the values present in 'TotalCharges' column and change the data type to numeric. 

In [ ]:
# Print the values in TotalCharges
df1.TotalCharges.unique()

As you can see total charges value is in string format. But there is one more interesting thing about these values. For 11 customers total charges value is blank!! To know more about it lets print those values.

In [ ]:
# Print rows with missing TotalCharges values
df1[df1.TotalCharges == ' ']

Note that for all above customers tenure is 0 months, that means they are new customers!! Total charges will be available for them after first month of service. Now instead of deleting these training examples we can add 0 as total charges!

In [ ]:
df1.TotalCharges =  df1.TotalCharges.replace(r' ', '0')
df1[df1.tenure == 0]

Now we have updated the missing value lets change the datatype of the total charges column to numeric.

In [ ]:
df1.TotalCharges = pd.to_numeric(df1.TotalCharges)
print(f'New datatype of TotalCharges : { df1.TotalCharges.dtype}')    

### Transform Categorical Values <a id ="12"></a>
* As you must have notice most of the columns have values 'Yes', 'No' , 'No phone service, 'No internet service' etc
* We will print all the unique values for categorical columns and then replace the duplicate categories like 'No phone service' and 'No internet service' with simple 'No' category.
* Once we get yes and no categories then, we will convert them  into numeric format (Yes: 1, No: 0)
* This is called as Hot Encoding. For more details about it please refer [One Hot Encoding](https://satishgunjal.com/one_hot_encoding/)

In [ ]:
def print_unique_col_values(df):
    """Print unique values from categorical columns of the given dataframe"""
    print('Unique values from categorical columns,\n')
    for column in df.columns:
        if(df[column].dtypes == 'object'): 
            print(f'column: {column}, Unique vlaues: {df[column].unique()}')
        
print_unique_col_values(df1)

Now lets replace 'No phone service' with 'No' in column MultipleLines and  ''No internet service' with 'no' in columns OnlineSecurity, DeviceProtection, TechSupport, StreamingTV, StreamingMovies. We can run this replaces, directly on the dataframe.

In [ ]:
df1.replace('No internet service', 'No', inplace = True)
df1.replace('No phone service', 'No', inplace = True)
# Lets print unique values again
print_unique_col_values(df1)

All our categorical values don't have any numeric order or relationship between them, means these 'Nominal  Categorical Values'. We should use One Hot Encoding instead of label encoding to convert them into numeric format. Since Churn contains our class labels we can manually assign numeric values to it.

In [ ]:
# Converting churn to numeric
df1['Churn'].replace({'Yes': 1,'No': 0},inplace=True)

In [ ]:
# Create df2 for cleaned dataset
df2 = pd.get_dummies(data = df1)

print(f'So we have added {df2.shape[1]- df1.shape[1]} more columns to our list. New shape : {df2.shape}')
df2.sample(5)

### Scaling & Normalization <a id ="13"></a>
In this step we are going identify the features who need to be scaled and scale them using sklearn's MinMaxScaler. Scaling is used to standardize the range of input features so that our machine learning model can quickly learn from the data.
In our case features tenure, MonthlyCharges and TotalCharges need to be scaled.

In [ ]:
cols_to_scale = ['tenure','MonthlyCharges','TotalCharges']

scaler = MinMaxScaler()
df2[cols_to_scale] = scaler.fit_transform(df2[cols_to_scale]) # Fit to data, then transform it
df2[cols_to_scale].describe()

As you can see now all our feature values are in the range of 0 to 1. Now let's prepare the training and testing data set for modeling.

# Train Test Split <a id ="14"></a>
We are going to create train and test datasets for training and testing respectively. Training dataset will have 80% of the data and test set will have 20% of the data. For more details please refer [link](https://satishgunjal.com/train_test_split/)

In [ ]:
# Create feature matrix X without label column 'Churn'
X = df2.drop('Churn',axis = 'columns')
# Create label vector y
y = df2['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 42)
print(f'X_train: {X_train.shape}, y_train: {y_train.shape}')
print(f'X_test: {X_test.shape}, y_test: {y_test.shape}')

# Lets have a look at our training datatset
X_train.sample(5)

# Modeling <a id ="15"></a>

## Model Building <a id ="16"></a>

Building the neural network model requires configuring the input, hidden and output layers. Since we have 38 input features we will create first layer with 38 neurons and 'relu' activation function. Then we will add the second dense layer with 14 nodes (or neurons) and ‘relu’ activation function. Since expected output is binary, in the last layer we add only one neuron with sigmoid activation function.

We will use 'binary_crossentropy' since the expected output is binary(churn or not).

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(38, input_shape= (38,), activation= 'relu'),
    tf.keras.layers.Dense(14, activation= 'relu'),
    tf.keras.layers.Dense(1, activation= 'sigmoid')     
])

model.compile(optimizer= 'adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

## Model Training <a id ="17"></a>
For model training we will run for 100 epochs.

In [ ]:
model.fit(X_train, y_train, epochs= 100)

## Model Evaluation <a id ="18"></a>
For model evaluation we will use test data.

In [ ]:
model.evaluate(X_test, y_test)

Now lets verify the model predictions on test data.

In [ ]:
predictions = model.predict(X_test)
predictions[:5]

As you can see from the results that predictions for the test data are in the form of 2D array with values ranging from 0 to 1. So in order to get in the binary format we will use threshold 0.5, anything more than 0.5 will be 1(churn-yes) else 0(churn-no)

In [ ]:
y_pred = []

for val in predictions:
    if val > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)
            
y_pred[:10]

Lets create a dataframe of true values and predicted values for comparison.

In [ ]:
df_true_pred = pd.DataFrame({'y_test':y_test, 'y_pred':y_pred}) 
df_true_pred[:10]

Check stats like precision, recall and f1-score

In [ ]:
print(classification_report(y_test,y_pred))

From the report we can see that,
* Precision(Accuracy of positive predictions), Recall(Fraction of positives that were correctly identified) and F1 score(What percent of positive predictions were correct) for 'class 0' is more than 80% and around 60% for 'class 1'
* The accuracy of the model was about 79% which is better than the 73.46%. Remember that 73.46% customer from given data do not churn.

Now let's print the confusion matrix for better visualization.

In [ ]:
cm = tf.math. confusion_matrix(labels= y_test, predictions= y_pred)
plt.figure(figsize = (10, 7))
sns.heatmap(cm, annot= True, fmt= 'd')
plt.xlabel('Predicted')
plt.ylabel('Truth')

# Conclusion <a id ="19"></a>
In EDA I could have gone more in detail for all the features and their effect on the customer churn. But I have only explained few major features which has the highest impact on customer attrition. Predicting customer churn is very tricky as there are so many external factors are also in play like what competition is doing and where does our product or service stand in the market. But all of these is possible to represent with data and hence good model can be built, tricky but possible!

# Reference <a id ="20"></a>
* https://www.youtube.com/watch?v=ocMd2loRfWE
* https://www.youtube.com/watch?v=MSBY28IJ47U&list=PLeo1K3hjS3uu7CxAacxVndI4bE_o3BDtO&index=18